<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkST0151ENSkillsNetwork20531532-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>



# **Final Project: Boston Housing**


##### Estimated time needed: **60** minutes


#### Import the required libraries we need for the lab.


In [ ]:
import piplite
await piplite.install(['numpy'],['pandas'])
await piplite.install(['seaborn'])

In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols


#### Read the dataset in the csv file from the URL


In [ ]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv'
resp = await fetch(URL)
boston_url = io.BytesIO((await resp.arrayBuffer()).to_py())

In [ ]:
boston_df=pd.read_csv(boston_url)

#### Add your code below following the instructions given in the course to complete the peer graded assignment


In [ ]:
# 1. Boxplot for MEDV variable (Median value of owner-occupied homes)
plt.figure(figsize=(8, 6))
sns.boxplot(y=boston_df['MEDV'])
plt.title('Boxplot of Median Value of Owner-Occupied Homes (MEDV)')
plt.ylabel('Median Value in $1000s')
plt.show()

# 2. Bar plot for the CHAS variable (Charles River dummy variable)
plt.figure(figsize=(8, 6))
chas_counts = boston_df['CHAS'].value_counts()
plt.bar(['Not Tract Bound River', 'Tract Bound River'], chas_counts)
plt.title('Bar Plot of Charles River Variable (CHAS)')
plt.ylabel('Count')
plt.show()

# 3. Boxplot for MEDV vs AGE (discretized)
boston_df['AGE_group'] = pd.cut(boston_df['AGE'], bins=[0, 35, 70, 100], 
                                 labels=['35 years and younger', 'Between 35 and 70 years', '70 years and older'])
plt.figure(figsize=(10, 6))
sns.boxplot(x='AGE_group', y='MEDV', data=boston_df)
plt.title('Median Home Values by Age Groups')
plt.xlabel('Age Groups')
plt.ylabel('Median Value in $1000s')
plt.show()

# 4. Scatter plot for NOX vs INDUS
plt.figure(figsize=(8, 6))
plt.scatter(boston_df['INDUS'], boston_df['NOX'], alpha=0.5)
plt.title('Scatter Plot: NOX vs INDUS')
plt.xlabel('Proportion of Non-Retail Business Acres (INDUS)')
plt.ylabel('Nitric Oxide Concentration (NOX)')
plt.show()

# 5. Histogram for PTRATIO
plt.figure(figsize=(8, 6))
plt.hist(boston_df['PTRATIO'], bins=20, edgecolor='black')
plt.title('Histogram of Pupil-Teacher Ratio (PTRATIO)')
plt.xlabel('Pupil-Teacher Ratio')
plt.ylabel('Frequency')
plt.show()

# Statistical Tests

# 1. Is there a significant difference in median value of houses bounded by the Charles river or not?
# T-test for MEDV between CHAS groups
chas_0 = boston_df[boston_df['CHAS'] == 0]['MEDV']
chas_1 = boston_df[boston_df['CHAS'] == 1]['MEDV']
t_stat, p_value = stats.ttest_ind(chas_0, chas_1)
print(f"\nT-test for MEDV by CHAS:")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")
if p_value < 0.05:
    print("Conclusion: There is a significant difference in median values between houses bounded by Charles river and not bounded.")
else:
    print("Conclusion: There is NO significant difference in median values between houses bounded by Charles river and not bounded.")

# 2. Is there a difference in Median values of houses (MEDV) for each proportion of owner occupied units built prior to 1940 (AGE)?
# ANOVA test for MEDV across AGE groups
anova_model = ols('MEDV ~ AGE_group', data=boston_df).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)
print(f"\nANOVA test for MEDV by AGE groups:")
print(anova_table)
if anova_table['PR(>F)'][0] < 0.05:
    print("Conclusion: There is a significant difference in median values across different AGE groups.")
else:
    print("Conclusion: There is NO significant difference in median values across different AGE groups.")

# 3. Can we conclude that there is no relationship between Nitric oxide concentrations and proportion of non-retail business acres per town?
# Pearson correlation test
pearson_corr, pearson_p = stats.pearsonr(boston_df['NOX'], boston_df['INDUS'])
print(f"\nPearson correlation between NOX and INDUS:")
print(f"Correlation coefficient: {pearson_corr:.4f}")
print(f"P-value: {pearson_p:.4f}")
if pearson_p < 0.05:
    print("Conclusion: There IS a statistically significant relationship between NOX and INDUS.")
else:
    print("Conclusion: There is NO statistically significant relationship between NOX and INDUS.")

# 4. What is the impact of an additional weighted distance to the five Boston employment centres on the median value of owner occupied homes?
# Regression analysis
X = boston_df['DIS']  # Weighted distances to five Boston employment centres
y = boston_df['MEDV']  # Median value of owner-occupied homes
X = sm.add_constant(X)  # Add constant for intercept
model = sm.OLS(y, X).fit()
print(f"\nRegression analysis: Impact of DIS on MEDV")
print(model.summary())
print("\nConclusion: For each unit increase in weighted distance to employment centers (DIS),")
print(f"the median home value changes by {model.params['DIS']:.4f} (in $1000s).")

# Clean up temporary column
boston_df = boston_df.drop('AGE_group', axis=1)

print("\nAnalysis completed!")